# HL SAOS: Frequency-Dependent Moduli

## Physical Context: Linear Response of Stress Distribution

In small-amplitude oscillatory shear (SAOS), the strain $\gamma(t) = \gamma_0 \sin(\omega t)$ is small enough that the stress distribution $P(\sigma, t)$ oscillates slightly around equilibrium without significant yielding.

**Glass phase ($\alpha < 0.5$):**
- **Low frequency ($\omega \tau \ll 1$):** Distribution has time to diffuse → G' approaches plateau, G'' ~ $\omega^\alpha$
- **Plateau modulus:** Reflects the frozen stress configuration, $G'_{plateau} \sim \sigma_c$
- **Power-law approach:** $G'(\omega) \sim \omega^{2\alpha}$ captures sub-diffusive relaxation
- **G'' peak:** Occurs near $\omega_c \sim 1/\tau$, marks transition to rubbery plateau

**Fluid phase ($\alpha \geq 0.5$):**
- **Terminal zone:** G' ~ $\omega^2$, G'' ~ $\omega$ (Maxwell-like)
- **No plateau:** G' → 0 as $\omega \to 0$ (complete stress relaxation)
- **Crossover:** G' = G'' at $\omega_c \sim 1/\tau$

**Key SAOS Features:**
- **Crossover frequency:** $\omega_c \approx 1/\tau$ → estimates microscopic relaxation time
- **Plateau height:** G'($\omega \to \infty$) ~ $\sigma_c$ → estimates yield threshold scale
- **Low-frequency slope:** $d\log G'/d\log \omega |_{\omega \to 0} \approx 2\alpha$ → estimates coupling parameter

**Cole-Cole Plot (G'' vs G'):**
- **Maxwell:** Perfect semicircle
- **HL Glass:** Skewed arc, reflects power-law modification
- **HL Fluid:** Arc with tail extending to origin

> **From the handbook:** "For small strains (linear regime), the HL model predicts: Glass (α < 0.5): G' approaches a plateau at low ω (frozen blocks); Fluid (α ≥ 0.5): G' → 0 as ω → 0 (terminal zone)."

## Learning Objectives

1. Understand small-amplitude oscillatory shear (SAOS) in the HL framework
2. Generate synthetic SAOS data from calibrated HL parameters
3. Fit G'(ω) and G''(ω) to extract HL parameters
4. Compare glass vs fluid phase frequency response
5. Analyze the Cole-Cole representation

## Prerequisites

- HL flow curve tutorial (hl/01_hl_flow_curve.ipynb) — parameters calibrated there
- Bayesian inference fundamentals (bayesian/01-bayesian-basics.ipynb)

## Runtime

- Fast demo (NUM_CHAINS=1, NUM_SAMPLES=500): ~2-3 minutes
- Full run (NUM_CHAINS=4, NUM_SAMPLES=2000): ~8-12 minutes

## 1. Setup

In [ ]:
# Google Colab setup
import sys

IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"
    print("RheoJAX installed successfully.")

In [ ]:
# Imports
%matplotlib inline
import json
import os
import sys
import time
import warnings

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.hl import HebraudLequeux

jax, jnp = safe_import_jax()
verify_float64()

FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

warnings.filterwarnings("ignore", message=".*jit.*|.*tracer.*", category=FutureWarning)
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")

## 2. Theory: SAOS in HL

In small-amplitude oscillatory shear (SAOS), a sinusoidal strain γ(t) = γ₀ sin(ωt) is applied and the stress response is measured:

$$
\sigma(t) = \gamma_0 [G'(\omega) \sin(\omega t) + G''(\omega) \cos(\omega t)]
$$

### HL SAOS Response

For small strains (linear regime), the HL model predicts:

**Storage modulus G'(ω)**: Elastic energy storage
- Glass (α < 0.5): G' approaches a plateau at low ω (frozen blocks)
- Fluid (α ≥ 0.5): G' → 0 as ω → 0 (terminal zone)

**Loss modulus G''(ω)**: Viscous dissipation
- Glass: G'' shows a peak related to yielding dynamics
- Fluid: G'' ∝ ω at low frequencies (viscous flow)

### Frequency Regimes

| ωτ | Response | Physics |
|----|----------|--------|
| ωτ << 1 | Terminal zone | Stress relaxes between oscillations |
| ωτ ≈ 1 | Crossover | G' ≈ G'', maximum energy dissipation |
| ωτ >> 1 | Plateau/rubbery | Stress blocks respond elastically |

### Parameters

- α: Controls whether G' plateaus (glass) or decays (fluid) at low ω
- τ: Sets the crossover frequency ω_c ≈ 1/τ
- σ_c: Scales the modulus magnitude

## 3. Load Calibrated Parameters

We use parameters calibrated from the flow curve (Notebook 01) to generate synthetic SAOS data.

In [ ]:
# Try to load calibrated parameters from NB 01
params_file = os.path.join("..", "outputs", "hl", "flow_curve", "nlsq_params_flow_curve.json")

if os.path.exists(params_file):
    with open(params_file) as f:
        calibrated_params = json.load(f)
    print("Loaded calibrated parameters from flow curve:")
else:
    # Fallback: use typical glass-phase parameters
    calibrated_params = {
        "alpha": 0.3,
        "tau": 1.0,
        "sigma_c": 100.0,
    }
    print("Using default glass-phase parameters:")

for name, val in calibrated_params.items():
    print(f"  {name:8s} = {val:.4g}")

# Classify phase
phase = "glass" if calibrated_params["alpha"] < 0.5 else "fluid"
print(f"\nPhase: {phase}")

## 4. Generate Synthetic SAOS Data

We generate G'(ω) and G''(ω) using simplified HL SAOS predictions with added noise.

In [ ]:
# Generate synthetic SAOS data
def generate_hl_saos(
    omega: np.ndarray,
    alpha: float,
    tau: float,
    sigma_c: float,
    noise_level: float = 0.03,
    seed: int = 42,
) -> tuple[np.ndarray, np.ndarray]:
    """Generate synthetic HL SAOS data.
    
    Uses simplified HL SAOS response (generalized Maxwell-like with HL modification).
    """
    rng = np.random.default_rng(seed)
    
    # Effective modulus scale
    G0 = sigma_c
    
    # Dimensionless frequency
    omega_tau = omega * tau
    
    # HL-modified response (simplified)
    # Glass (α < 0.5): G' approaches plateau, power-law modification
    # Fluid (α ≥ 0.5): Standard Maxwell-like terminal behavior
    
    if alpha < 0.5:
        # Glass: plateau at low frequency, power-law transition
        G_prime = G0 * omega_tau ** (2 * alpha) / (1 + omega_tau ** (2 * alpha))
        G_double_prime = G0 * omega_tau ** alpha / (1 + omega_tau ** (2 * alpha))
    else:
        # Fluid: terminal zone (Maxwell-like)
        G_prime = G0 * omega_tau ** 2 / (1 + omega_tau ** 2)
        G_double_prime = G0 * omega_tau / (1 + omega_tau ** 2)
    
    # Add noise
    noise_p = rng.normal(0, noise_level * np.mean(G_prime), size=G_prime.shape)
    noise_pp = rng.normal(0, noise_level * np.mean(G_double_prime), size=G_double_prime.shape)
    
    return G_prime + noise_p, G_double_prime + noise_pp

# Generate data
omega = np.logspace(-2, 2, 50)  # 0.01 to 100 rad/s
G_prime, G_double_prime = generate_hl_saos(
    omega,
    calibrated_params["alpha"],
    calibrated_params["tau"],
    calibrated_params["sigma_c"],
)

print(f"Generated {len(omega)} SAOS points")
print(f"ω range: [{omega.min():.2f}, {omega.max():.1f}] rad/s")

In [ ]:
# Plot synthetic SAOS data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# G' and G'' vs frequency
ax1.loglog(omega, G_prime, "o", markersize=5, label="G' (storage)")
ax1.loglog(omega, G_double_prime, "s", markersize=5, label="G'' (loss)")
ax1.axvline(1/calibrated_params["tau"], color="gray", linestyle="--", alpha=0.5, label=f"ω_c = 1/τ")
ax1.set_xlabel("Frequency ω [rad/s]")
ax1.set_ylabel("Modulus [Pa]")
ax1.set_title(f"SAOS Moduli ({phase} phase, α={calibrated_params['alpha']:.2f})")
ax1.legend()
ax1.grid(True, alpha=0.3, which="both")

# Cole-Cole plot (G'' vs G')
ax2.plot(G_prime, G_double_prime, "o-", markersize=5)
ax2.set_xlabel("G' [Pa]")
ax2.set_ylabel("G'' [Pa]")
ax2.set_title("Cole-Cole Plot")
ax2.set_aspect("equal")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Glass vs Fluid Comparison

Let's compare SAOS response for glass (α < 0.5) vs fluid (α ≥ 0.5) phases.

In [ ]:
# Generate SAOS for different α values
alpha_values = [0.2, 0.4, 0.6, 0.8]
tau_fixed = calibrated_params["tau"]
sigma_c_fixed = calibrated_params["sigma_c"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
colors = plt.cm.viridis(np.linspace(0.2, 0.8, len(alpha_values)))

for i, alpha in enumerate(alpha_values):
    G_p, G_pp = generate_hl_saos(omega, alpha, tau_fixed, sigma_c_fixed, noise_level=0)
    phase_label = "glass" if alpha < 0.5 else "fluid"
    
    ax1.loglog(omega, G_p, "-", color=colors[i], lw=2, label=f"α={alpha:.1f} ({phase_label})")
    ax2.loglog(omega, G_pp, "-", color=colors[i], lw=2, label=f"α={alpha:.1f}")

ax1.axvline(1/tau_fixed, color="gray", linestyle="--", alpha=0.5)
ax1.set_xlabel("Frequency ω [rad/s]")
ax1.set_ylabel("G' [Pa]")
ax1.set_title("Storage Modulus: Glass vs Fluid")
ax1.legend(fontsize=9)
ax1.grid(True, alpha=0.3, which="both")

ax2.axvline(1/tau_fixed, color="gray", linestyle="--", alpha=0.5)
ax2.set_xlabel("Frequency ω [rad/s]")
ax2.set_ylabel("G'' [Pa]")
ax2.set_title("Loss Modulus: Glass vs Fluid")
ax2.legend(fontsize=9)
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

### Key Observations

1. **Glass (α < 0.5)**: G' shows power-law approach to plateau at low ω
2. **Fluid (α ≥ 0.5)**: G' → ω² at low frequency (Maxwell-like terminal zone)
3. **Crossover frequency**: ω_c ≈ 1/τ where G' ≈ G''
4. **α controls the low-frequency slope**: Lower α → slower approach to plateau

## 6. NLSQ Fitting

Fit the HL model to the synthetic SAOS data to recover parameters.

In [ ]:
# For SAOS fitting, we need to create complex modulus data
# G* = G' + i*G''
# We'll stack G' and G'' for fitting

model = HebraudLequeux()

# Create combined data array (G' followed by G'')
# Note: This is a simplified approach - a full implementation would
# fit the complex modulus directly

# For this tutorial, we demonstrate the concept using the flow curve mode
# as the HL model's SAOS implementation would need the oscillation kernel

# Instead, we'll demonstrate parameter recovery using the known relationship
# and validate against the true parameters

print("True parameters (used to generate data):")
for name, val in calibrated_params.items():
    print(f"  {name:8s} = {val:.4g}")

# Extract key features from SAOS data to estimate parameters
# 1. Crossover frequency → τ estimate
# 2. High-frequency plateau → σ_c estimate
# 3. Low-frequency slope → α estimate

# Find crossover (G' ≈ G'')
crossover_idx = np.argmin(np.abs(G_prime - G_double_prime))
omega_c_est = omega[crossover_idx]
tau_est = 1.0 / omega_c_est

# High-frequency modulus
G_plateau_est = np.mean(G_prime[-5:])
sigma_c_est = G_plateau_est

# Low-frequency slope (for α < 0.5, slope ≈ 2α)
low_freq_mask = omega < omega_c_est / 5
# Filter positive values to avoid log10 of noise-induced negatives
pos_mask = low_freq_mask & (G_prime > 0)
if np.sum(pos_mask) >= 3:
    slope = np.polyfit(np.log10(omega[pos_mask]), np.log10(G_prime[pos_mask]), 1)[0]
    alpha_est = slope / 2  # G' ~ ω^(2α) for glass
    alpha_est = np.clip(alpha_est, 0.1, 0.9)
else:
    alpha_est = 0.3

print("\nEstimated parameters from SAOS features:")
print(f"  α (from slope)      = {alpha_est:.3f} (true: {calibrated_params['alpha']:.3f})")
print(f"  τ (from crossover)  = {tau_est:.3f} (true: {calibrated_params['tau']:.3f})")
print(f"  σ_c (from plateau)  = {sigma_c_est:.1f} (true: {calibrated_params['sigma_c']:.1f})")

In [ ]:
# Plot data with estimated parameters
G_p_est, G_pp_est = generate_hl_saos(omega, alpha_est, tau_est, sigma_c_est, noise_level=0)

fig, ax = plt.subplots(figsize=(10, 6))

ax.loglog(omega, G_prime, "o", markersize=6, color="C0", label="G' data")
ax.loglog(omega, G_double_prime, "s", markersize=6, color="C1", label="G'' data")
ax.loglog(omega, G_p_est, "-", lw=2, color="C0", alpha=0.7, label="G' fit")
ax.loglog(omega, G_pp_est, "-", lw=2, color="C1", alpha=0.7, label="G'' fit")

ax.axvline(omega_c_est, color="gray", linestyle="--", alpha=0.5, label=f"ω_c = {omega_c_est:.2f}")
ax.set_xlabel("Frequency ω [rad/s]")
ax.set_ylabel("Modulus [Pa]")
ax.set_title(f"SAOS Fit: α={alpha_est:.2f}, τ={tau_est:.2f}, σ_c={sigma_c_est:.0f}")
ax.legend()
ax.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 7. Bayesian Parameter Estimation

For a complete Bayesian analysis, we would use the HL model's `fit_bayesian` with `test_mode='oscillation'`. Here we demonstrate the concept using the estimated parameters as priors.

In [ ]:
# Compute uncertainties using simple bootstrap
n_bootstrap = 500
rng = np.random.default_rng(42)

alpha_samples = []
tau_samples = []
sigma_c_samples = []

for _ in range(n_bootstrap):
    # Add noise and re-estimate
    noise = rng.normal(0, 0.03 * np.mean(G_prime), size=G_prime.shape)
    G_p_boot = G_prime + noise
    G_pp_boot = G_double_prime + noise
    
    # Re-estimate crossover
    cross_idx = np.argmin(np.abs(G_p_boot - G_pp_boot))
    omega_c_boot = omega[cross_idx]
    tau_boot = 1.0 / omega_c_boot
    
    # Re-estimate plateau
    sigma_c_boot = np.mean(G_p_boot[-5:])
    
    # Re-estimate slope (filter positive values to avoid log10 of negatives)
    pos_mask_boot = low_freq_mask & (G_p_boot > 0)
    if np.sum(pos_mask_boot) >= 3:
        slope_boot = np.polyfit(np.log10(omega[pos_mask_boot]), np.log10(G_p_boot[pos_mask_boot]), 1)[0]
        alpha_boot = np.clip(slope_boot / 2, 0.05, 0.95)
    else:
        alpha_boot = alpha_est
    
    alpha_samples.append(alpha_boot)
    tau_samples.append(tau_boot)
    sigma_c_samples.append(sigma_c_boot)

alpha_samples = np.array(alpha_samples)
tau_samples = np.array(tau_samples)
sigma_c_samples = np.array(sigma_c_samples)

In [ ]:
# Compute P(glass)
p_glass = np.mean(alpha_samples < 0.5)
print(f"P(glass) = P(α < 0.5) = {p_glass:.1%}")

# Plot posterior histogram
fig, ax = plt.subplots(figsize=(9, 5))

ax.hist(alpha_samples, bins=50, density=True, alpha=0.7, color="C0", edgecolor="black")
ax.axvline(0.5, color="red", linestyle="--", linewidth=2, label="Glass transition (α=0.5)")
ax.axvline(calibrated_params["alpha"], color="green", linestyle="-", linewidth=2, label=f"True α={calibrated_params['alpha']:.2f}")

ax.axvspan(0, 0.5, alpha=0.15, color="red", label=f"Glass (P={p_glass:.1%})")
ax.axvspan(0.5, 1.0, alpha=0.15, color="blue", label=f"Fluid (P={1-p_glass:.1%})")

ax.set_xlabel("α (coupling parameter)")
ax.set_ylabel("Bootstrap density")
ax.set_title(f"SAOS-Based Phase Classification: P(glass) = {p_glass:.1%}")
ax.legend()
ax.set_xlim(0, 1)

plt.tight_layout()
display(fig)
plt.close(fig)

In [ ]:
# Parameter summary
print("Parameter Estimates (Bootstrap)")
print("=" * 65)
print(f"{'Param':>10s}  {'True':>12s}  {'Median':>12s}  {'95% CI':>24s}")
print("-" * 65)

for name, true_val, samples in [
    ("alpha", calibrated_params["alpha"], alpha_samples),
    ("tau", calibrated_params["tau"], tau_samples),
    ("sigma_c", calibrated_params["sigma_c"], sigma_c_samples),
]:
    median = np.median(samples)
    lo = np.percentile(samples, 2.5)
    hi = np.percentile(samples, 97.5)
    print(f"{name:>10s}  {true_val:12.4g}  {median:12.4g}  [{lo:.4g}, {hi:.4g}]")

## 8. Save Results

In [ ]:
# Save results
output_dir = os.path.join("..", "outputs", "hl", "saos")
os.makedirs(output_dir, exist_ok=True)

# Save estimated parameters
saos_params = {
    "alpha": float(alpha_est),
    "tau": float(tau_est),
    "sigma_c": float(sigma_c_est),
}
with open(os.path.join(output_dir, "nlsq_params_saos.json"), "w") as f:
    json.dump(saos_params, f, indent=2)

# Save bootstrap samples (pseudo-posterior)
posterior_dict = {
    "alpha": alpha_samples.tolist(),
    "tau": tau_samples.tolist(),
    "sigma_c": sigma_c_samples.tolist(),
}
with open(os.path.join(output_dir, "posterior_saos.json"), "w") as f:
    json.dump(posterior_dict, f)

print(f"Results saved to {output_dir}/")
print(f"  nlsq_params_saos.json: {len(saos_params)} parameters")
print(f"  posterior_saos.json: {len(alpha_samples)} bootstrap draws")

## Key Takeaways

1. **SAOS probes linear viscoelasticity** — G'(ω) and G''(ω) reveal the frequency-dependent response without large deformations.

2. **Glass vs fluid signatures**:
   - Glass (α < 0.5): G' plateau at low ω, power-law approach with exponent 2α
   - Fluid (α ≥ 0.5): G' ~ ω² at low ω (Maxwell terminal zone)

3. **Key SAOS features**:
   - Crossover frequency ω_c ≈ 1/τ → estimates relaxation time
   - High-frequency plateau → estimates σ_c (elastic modulus scale)
   - Low-frequency slope → estimates α (phase parameter)

4. **Cole-Cole plot** (G'' vs G') reveals:
   - Semicircle: Single relaxation time (Maxwell)
   - Skewed arc: Distribution of relaxation times
   - HL: Power-law modification of Maxwell response

5. **SAOS limitations for HL**:
   - Small-amplitude limit may not probe yielding fully
   - α estimation requires low-frequency data (long experiments)
   - Combine with flow curve for robust parameter estimation

### Next Steps

- **NB 05**: Startup shear (stress overshoot dynamics)
- **NB 06**: LAOS (nonlinear oscillatory response)

## Further Reading

- [Hébraud-Lequeux Model Handbook](../../docs/source/models/hl/hebraud_lequeux.rst)
- Hébraud, P. & Lequeux, F. (1998). Mode-coupling theory for the pasty rheology of soft glassy materials. *Phys. Rev. Lett.*, 81, 2934. https://doi.org/10.1103/PhysRevLett.81.2934
- Fielding, S. M., Sollich, P., & Cates, M. E. (2000). Aging and rheology in soft materials. *J. Rheol.*, 44(2), 323-369. https://doi.org/10.1122/1.551088
- Sollich, P., Lequeux, F., Hébraud, P., & Cates, M. E. (1997). Rheology of soft glassy materials. *Phys. Rev. Lett.*, 78, 2020. https://doi.org/10.1103/PhysRevLett.78.2020